This notebook is to create the submission.csv for the kaggle competion

In [ ]:
from src import ds_generator, models
import tensorflow as tf
import util
import os
import tqdm
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

In [1]:
solutions_folder = "path_for_our_solutions"
distance_for_new_whales = 0.35

In [ ]:
test_df = pd.DataFrame(os.listdir(util.TEST_DATA_PATH),columns=["image"])
test_df["label"]=0
test_ds = ds_generator.DS_Generator().return_plain_ds(test_df,64,train_imgs = False)
whole_train_ds = ds_generator.DS_Generator().return_plain_ds(util.TRAIN_SPECIES_DF,64)

In [ ]:
# some model you want to try
model = models.return_soft_max_pretrained_siamese_model()

In [ ]:
train_embedding = model.predict(whole_train_ds,verbose=1)
test_embedding = model.predict(test_ds,verbose=1)

# save just in case
np.save(solutions_folder + "/train_embedding.npy",train_embedding)
np.save(solutions_folder + "/test_embedding.npy",test_embedding )


# compute the pairwise_distance (this will take long)
pairwise_distance = cdist(test_embedding,train_embedding) # distance_‘cosine’
# and also save it just in case
np.save(solutions_folder + "/pairwise_distance.npy",pairwise_distance )

# split it up to not overstress memory and then calculate best 5 vals/indexes
split_ups = np.ceil(np.linspace(0,27956)).astype(int)
closest_vals, closest_indices = list(),list()
for i in tqdm.tqdm(range(1,len(split_ups))):
    split = pairwise_distance[split_ups[i-1]:split_ups[i]]
    vals , indexes = tf.nn.top_k(tf.math.negative(split),5)
    vals = tf.math.negative(vals)
    closest_vals.append(vals)
    closest_indices.append(indexes)

# re-concatenate
best_vals = np.concatenate(closest_vals)
best_indexes = np.concatenate(closest_indices)

for i in tqdm.tqdm(range(len(test_df))):
    distances = best_vals [i]
    labels = [util.TRAIN_SPECIES_DF.iloc[k,2] for k in best_indexes[i]]

    # if the distance to a neighboring whale is bigger then the distance for new whales, insert a new whale
    args = np.argwhere(distances > distance_for_new_whales)
    if len(args)!=0:
        first = int(args[0,0])
        labels.insert(first,"new_individual")

    test_df.iloc[i,-1]= " ".join(labels[:5])

# print submission.csv
test_df.to_csv(solutions_folder + "/submission.csv",index=False,columns=["image","predictions"])